In [ ]:
# !pip install -q -U jax jaxlib
!pip install -q -U flax
!pip install -q sentence-transformers
#!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q transformers
!pip install -q torch torchvision
!pip install -q comet_ml==3.12.2
#!pip install -r drive/MyDrive/ColabNotebooks/hybrid_clip/requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
import os 
import sys
import json
import zipfile
import natsort

import numpy as np
import pandas as pd
from tqdm import tqdm
import urllib.request

from IPython.display import display, Image

from PIL import Image as PilImage

os.environ['TOKENIZERS_PARALLELISM'] = "false"

import transformers
from transformers import AutoTokenizer

import torch
import torchvision
from torchvision import transforms
from torchvision.transforms import CenterCrop, ConvertImageDtype, Normalize, Resize, ToTensor
from torchvision.transforms.functional import InterpolationMode

In [ ]:
#from modeling_hybrid_clip import FlaxHybridCLIP
from transformers import FlaxVisionTextDualEncoderModel

In [ ]:
PROJ_DIR= "drive/MyDrive/ColabNotebooks/"
TRN_IMG_ZIP_FOLDER = PROJ_DIR+ "data/CLIP/train_v2/trn_img_tags_v2_100k_p2.zip"
VALID_IMG_ZIP_FOLDER = PROJ_DIR+ "data/CLIP/eval_v2/eval_img_tags_v2_10k.zip"

In [ ]:
SCRIPT_DIR = "drive/MyDrive/ColabNotebooks/hybrid_clip"
MODEL_DIR = "drive/MyDrive/ColabNotebooks/models/clip-finetuned"
IMAGE_ENCODER="openai/clip-vit-base-patch32"
TEXT_ENCODER="cardiffnlp/twitter-roberta-base-jun2022"

TRAIN_FILE= PROJ_DIR+"data/CLIP/train_v2/trn_img_tags_v2_100k_p2.json"
VALID_FILE= PROJ_DIR+"data/CLIP/eval_v2/valid_img_tags_v2_10k.json"
IMG_FOLDER = './images'
VALID_IMG_FOLDER = './valid_images'

### Setup images train and valid data folder

In [ ]:
## Unzip images folder
if not os.path.exists(IMG_FOLDER) or len(os.listdir(IMG_FOLDER)) == 0:
    os.makedirs(IMG_FOLDER, exist_ok=True)

with zipfile.ZipFile(TRN_IMG_ZIP_FOLDER, 'r') as zf:
    for member in tqdm(zf.infolist(), desc='Extracting'):
        zf.extract(member, IMG_FOLDER)

Extracting: 100%|██████████| 99079/99079 [01:50<00:00, 893.01it/s]


In [ ]:
## Unzip valid images
if not os.path.exists(VALID_IMG_FOLDER) or len(os.listdir(VALID_IMG_FOLDER)) == 0:
    os.makedirs(VALID_IMG_FOLDER, exist_ok=True)
with zipfile.ZipFile(VALID_IMG_ZIP_FOLDER, 'r') as zf:
    for member in tqdm(zf.infolist(), desc='Extracting'):
        zf.extract(member, VALID_IMG_FOLDER)

Extracting: 100%|██████████| 11896/11896 [00:24<00:00, 489.83it/s]


### Call train script

In [ ]:
!export COMET_API_KEY=""
!python -W ignore $SCRIPT_DIR/run_hybrid_clip.py \
    --output_dir $MODEL_DIR \
    --overwrite_output_dir \
    --tokenizer_name=$TEXT_ENCODER \
    --train_file=$TRAIN_FILE \
    --validation_file=$VALID_FILE \
    --do_train --do_eval \
    --num_train_epochs="100" --max_seq_length 24 \
    --per_device_train_batch_size="128" \
    --per_device_eval_batch_size="128" \
    --learning_rate="0.0001" --warmup_ratio 0.1 --weight_decay 0.0 \
    --preprocessing_num_workers 8 \
    --exp_name clip_finetune_v2 \
    --eval_when 1 \
    --text_model_name_or_path=$TEXT_ENCODER \
    --vision_model_name_or_path=$IMAGE_ENCODER \
    --log_comet \
    --epoch_offset 44 \
    --run_from_checkpoint $FROM_CHECKPOINT
#    --freeze_backbones \ #Try unfreeze backbones